In [0]:
%sql
USE CATALOG de_project_catalog;
USE SCHEMA olist_silver;

In [0]:
Bronze_base_path = "/Volumes/de_project_catalog/olist_bronze/raw_data/bronze"
Silver_base_path = "/Volumes/de_project_catalog/olist_silver/cleaned_data"

### Orders

In [0]:
bronze_orders_df = spark.read.parquet(
   f"{Bronze_base_path}/orders"
)

bronze_orders_df.printSchema()
bronze_orders_df.show(5)

In [0]:
bronze_orders_df.filter("order_id is Null").count()

In [0]:
bronze_orders_df.groupBy("order_id").count().filter("count > 1").count()

In [0]:
corrupt_record_df  = bronze_orders_df.filter("corrupt_data is Not Null")
corrupt_record_df.display()

In [0]:
bronze_orders_df.display()

In [0]:
from pyspark.sql.functions import row_number, desc
from pyspark.sql.window import Window
orders_clean_df =(
     bronze_orders_df
     .filter("order_id is Not Null")
     .withColumn("rn", row_number().over(Window.partitionBy("order_id").orderBy(desc("order_purchase_timestamp"))))
     .filter("rn == 1")
     .drop("rn", "corrupt_data")
)
orders_clean_df.display()